Intialization

In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots

Data Cleaning

In [2]:
#load dataset
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/metacritic game/games.csv')
df

,id,title,releaseDate,rating,genres,description,platforms,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,platform_metascores,developer,publisher
0,1300001290,The Legend of Zelda: Ocarina of Time,1998-11-23,E,Open-World Action,"As a young boy, Link is tricked by Ganondorf, ...",Nintendo 64,99.0,22.0,Universal acclaim,91.0,10611,Universal acclaim,99,Nintendo,"Nintendo,Gradiente"
1,1300001928,SoulCalibur,1999-09-08,T,3D Fighting,"[Xbox Live Arcade] Soulcalibur, the highest M...","Dreamcast,iOS (iPhone/iPad),Xbox 360",98.0,24.0,Universal acclaim,78.0,605,Generally favorable,"98,73,79",Namco,Namco
2,1300027043,Grand Theft Auto IV,2008-04-29,M,Open-World Action,[Metacritic's 2008 PS3 Game of the Year; Also ...,"PlayStation 3,Xbox 360,PC",98.0,86.0,Universal acclaim,83.0,4781,Generally favorable,"98,98,90",Rockstar North,"Rockstar Games,Capcom"
3,1300019237,Super Mario Galaxy,2007-11-12,E,3D Platformer,[Metacritic's 2007 Wii Game of the Year] The u...,Wii,97.0,73.0,Universal acclaim,91.0,4181,Universal acclaim,97,Nintendo,Nintendo
4,1300044129,Super Mario Galaxy 2,2010-05-23,E,3D Platformer,"Super Mario Galaxy 2, the sequel to the galaxy...",Wii,97.0,87.0,Universal acclaim,90.0,2933,Universal acclaim,97,Nintendo EAD Tokyo,"Nintendo,iQue"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13437,1300035158,SPOGS Racing,2008-07-07,E,Auto Racing,[WiiWare] Race along in distinctively unique ...,Wii,18.0,7.0,Overwhelming dislike,61.0,10,Mixed or average,18,Pronto Games,D2C Games
13438,1300156575,Double Dragon II: Wander of the Dragons,2013-04-05,T,2D Beat-'Em-Up,A remake the 80's Double Dragon II game. Team ...,Xbox 360,17.0,19.0,Overwhelming dislike,16.0,66,Overwhelming dislike,17,Gravity,CFK
13439,1300465525,Vroom in the Night Sky,2017-04-05,E,Biking,Vroom in the night sky is a magical bike actio...,Nintendo Switch,17.0,15.0,Overwhelming dislike,33.0,124,Generally unfavorable,17,Poisoft,Poisoft
13440,1300033704,Yaris,2007-10-10,E10+,Auto Racing,[Xbox Live Arcade] Hop into a Toyota Yaris an...,Xbox 360,17.0,7.0,Overwhelming dislike,51.0,61,Mixed or average,17,Backbone Entertainment,Backbone Entertainment


In [3]:
#check duplicated rows
df[df.duplicated()]             #no duplicate records found

,id,title,releaseDate,rating,genres,description,platforms,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,platform_metascores,developer,publisher


In [4]:
#identify nulls in all columns
df.isna().sum()                 #for rating it is ok to have NA, in EDA make sure to filter NAs out
                                #since these columns are mainly used for EDA, and no further computations
                                #however, for study purposes, NA records that are minimal will be eliminated
                                #columns with high NA count will be handled via distribution percentage

df = df.dropna(subset=['releaseDate','platforms','metascore','metascore_count','metascore_sentiment', 
                  'platform_metascores','developer','publisher'])

                                #since description is not used in EDA, the records with NAs are left as it is

#handling 'rating' column
df.loc[:, 'rating'] = df['rating'].fillna('unknown')   
                                #fill with 'unknown' which can later be filtered out during EDA

#handling 'userscore_sentiment' column
df['userscore_sentiment'].nunique()             #only five categorical sentiments
df['userscore_sentiment'].value_counts(normalize=True) * 100      
                                #check distribution percentage
                                #since only two of the sentiments are dominant, assign randomly to NAs 
np.random.seed(42)              #reproducable randoms
mask = df['userscore_sentiment'].isna()
df.loc[mask,'userscore_sentiment'] = np.random.choice(['Mixed or average','Generally favorable'], size=mask.sum())
df.isna().sum() 

id                      0
title                   0
releaseDate             0
rating                  0
genres                  0
description            47
platforms               0
metascore               0
metascore_count         0
metascore_sentiment     0
userscore               0
userscore_count         0
userscore_sentiment     0
platform_metascores     0
developer               0
publisher               0
dtype: int64

In [5]:
temp_df = df.copy()

#trim all object data type in dataframe
obj_cols = temp_df.select_dtypes(include='object').columns
temp_df[obj_cols] = temp_df[obj_cols].apply(lambda x: x.str.strip())

#convert column datatypes float to int
num_cols = ['metascore','metascore_count','userscore']
temp_df[num_cols] = temp_df[num_cols].astype('int64')

#convert column datatype for date
temp_df['releaseDate'] = pd.to_datetime(temp_df['releaseDate'])
temp_df

,id,title,releaseDate,rating,genres,description,platforms,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,platform_metascores,developer,publisher
0,1300001290,The Legend of Zelda: Ocarina of Time,1998-11-23,E,Open-World Action,"As a young boy, Link is tricked by Ganondorf, ...",Nintendo 64,99,22,Universal acclaim,91,10611,Universal acclaim,99,Nintendo,"Nintendo,Gradiente"
1,1300001928,SoulCalibur,1999-09-08,T,3D Fighting,"[Xbox Live Arcade] Soulcalibur, the highest M...","Dreamcast,iOS (iPhone/iPad),Xbox 360",98,24,Universal acclaim,78,605,Generally favorable,"98,73,79",Namco,Namco
2,1300027043,Grand Theft Auto IV,2008-04-29,M,Open-World Action,[Metacritic's 2008 PS3 Game of the Year; Also ...,"PlayStation 3,Xbox 360,PC",98,86,Universal acclaim,83,4781,Generally favorable,"98,98,90",Rockstar North,"Rockstar Games,Capcom"
3,1300019237,Super Mario Galaxy,2007-11-12,E,3D Platformer,[Metacritic's 2007 Wii Game of the Year] The u...,Wii,97,73,Universal acclaim,91,4181,Universal acclaim,97,Nintendo,Nintendo
4,1300044129,Super Mario Galaxy 2,2010-05-23,E,3D Platformer,"Super Mario Galaxy 2, the sequel to the galaxy...",Wii,97,87,Universal acclaim,90,2933,Universal acclaim,97,Nintendo EAD Tokyo,"Nintendo,iQue"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13437,1300035158,SPOGS Racing,2008-07-07,E,Auto Racing,[WiiWare] Race along in distinctively unique ...,Wii,18,7,Overwhelming dislike,61,10,Mixed or average,18,Pronto Games,D2C Games
13438,1300156575,Double Dragon II: Wander of the Dragons,2013-04-05,T,2D Beat-'Em-Up,A remake the 80's Double Dragon II game. Team ...,Xbox 360,17,19,Overwhelming dislike,16,66,Overwhelming dislike,17,Gravity,CFK
13439,1300465525,Vroom in the Night Sky,2017-04-05,E,Biking,Vroom in the night sky is a magical bike actio...,Nintendo Switch,17,15,Overwhelming dislike,33,124,Generally unfavorable,17,Poisoft,Poisoft
13440,1300033704,Yaris,2007-10-10,E10+,Auto Racing,[Xbox Live Arcade] Hop into a Toyota Yaris an...,Xbox 360,17,7,Overwhelming dislike,51,61,Mixed or average,17,Backbone Entertainment,Backbone Entertainment


In [6]:
#standardize of values in columns
#handling 'rating' column
def get_ratings(category):
    if category == 'E' or category == 'K-A':
        return 'Everyone'
    elif category == 'T':
        return 'Teen'
    elif category == 'AO':
        return 'Adults Only'
    elif category == 'E10+':
        return 'Everyone (Above 10)'
    elif category == 'M':
        return 'Mature'
    elif category == 'RP' or category == 'unknown':
        return 'Rating Pending'
    
def normalize_platforms(platforms):
    normalized = []
    for p in platforms:
        if 'PlayStation' in p:
            normalized.append('PlayStation')
        elif 'Xbox' in p:
            normalized.append('Xbox')
        elif p in ['3DS','2DS','DS','Wii','Wii U'] or 'Nintendo' in p:
            normalized.append('Nintendo')
        else:
            normalized.append(p)
    return normalized

temp_df['rating'] = df['rating'].apply(get_ratings)

#handling 'publisher' column
temp_df['publisher'] = df['publisher'].str.split(r',\s*').str[0]     #keep only main publisher

#handling 'platform' column
temp_df['platform_list'] = df['platforms'].str.split(r',\s*')
temp_df['platform_list'] = temp_df['platform_list'].apply(normalize_platforms)

#handling 'platform_metascores' column
temp_df['score_list'] = df['platform_metascores'].astype(str).str.split(r',\s*')
df_exploded = temp_df.explode(['platform_list','score_list',])
df_exploded       

,id,title,releaseDate,rating,genres,description,platforms,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,platform_metascores,developer,publisher,platform_list,score_list
0,1300001290,The Legend of Zelda: Ocarina of Time,1998-11-23,Everyone,Open-World Action,"As a young boy, Link is tricked by Ganondorf, ...",Nintendo 64,99,22,Universal acclaim,91,10611,Universal acclaim,99,Nintendo,Nintendo,Nintendo,99
1,1300001928,SoulCalibur,1999-09-08,Teen,3D Fighting,"[Xbox Live Arcade] Soulcalibur, the highest M...","Dreamcast,iOS (iPhone/iPad),Xbox 360",98,24,Universal acclaim,78,605,Generally favorable,"98,73,79",Namco,Namco,Dreamcast,98
1,1300001928,SoulCalibur,1999-09-08,Teen,3D Fighting,"[Xbox Live Arcade] Soulcalibur, the highest M...","Dreamcast,iOS (iPhone/iPad),Xbox 360",98,24,Universal acclaim,78,605,Generally favorable,"98,73,79",Namco,Namco,iOS (iPhone/iPad),73
1,1300001928,SoulCalibur,1999-09-08,Teen,3D Fighting,"[Xbox Live Arcade] Soulcalibur, the highest M...","Dreamcast,iOS (iPhone/iPad),Xbox 360",98,24,Universal acclaim,78,605,Generally favorable,"98,73,79",Namco,Namco,Xbox,79
2,1300027043,Grand Theft Auto IV,2008-04-29,Mature,Open-World Action,[Metacritic's 2008 PS3 Game of the Year; Also ...,"PlayStation 3,Xbox 360,PC",98,86,Universal acclaim,83,4781,Generally favorable,"98,98,90",Rockstar North,Rockstar Games,PlayStation,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13437,1300035158,SPOGS Racing,2008-07-07,Everyone,Auto Racing,[WiiWare] Race along in distinctively unique ...,Wii,18,7,Overwhelming dislike,61,10,Mixed or average,18,Pronto Games,D2C Games,Nintendo,18
13438,1300156575,Double Dragon II: Wander of the Dragons,2013-04-05,Teen,2D Beat-'Em-Up,A remake the 80's Double Dragon II game. Team ...,Xbox 360,17,19,Overwhelming dislike,16,66,Overwhelming dislike,17,Gravity,CFK,Xbox,17
13439,1300465525,Vroom in the Night Sky,2017-04-05,Everyone,Biking,Vroom in the night sky is a magical bike actio...,Nintendo Switch,17,15,Overwhelming dislike,33,124,Generally unfavorable,17,Poisoft,Poisoft,Nintendo,17
13440,1300033704,Yaris,2007-10-10,Everyone (Above 10),Auto Racing,[Xbox Live Arcade] Hop into a Toyota Yaris an...,Xbox 360,17,7,Overwhelming dislike,51,61,Mixed or average,17,Backbone Entertainment,Backbone Entertainment,Xbox,17


EDA Section 

In [7]:
#Time Based Analysis
#since dataset only has 'releasedate' up to march 14, 2025, the EDA study will use data up to 2025 year only
#and also drop below year 2000 as the records are low during this period

case_df = df_exploded.copy()
case_df = case_df.drop_duplicates(subset='id')
case_df['Year'] = case_df['releaseDate'].dt.year
case_df = case_df[(case_df['Year'] >= 2000) & (case_df['Year'] != 2025)]

#finding no of games released per year
games_no_df = case_df.groupby('Year')['id'].count().reset_index(name='count')
games_no_df['increase'] = games_no_df['count'].diff().fillna(0)
games_no_df['increase'] = games_no_df['increase'].astype(int)

fig = px.line(games_no_df, x='Year', y='count', markers=True, title='<b>No.of Games Released from Year 2000 to 2024</b>',
                labels={'count':'No. of Games'})
fig.update_traces(line_width=2, line_color='darkblue')

#getting the year with most and least no.of games released
max_count = games_no_df['count'].idxmax()
min_count = games_no_df['count'].idxmin()

#getting the year with highest increase/decrease from previous year
max_inc = games_no_df['increase'].idxmax()
max_dec = games_no_df['increase'].idxmin()

fig.add_annotation(
    x = games_no_df.loc[max_count, 'Year'],
    y = games_no_df.loc[max_count, 'count'],
    text = f'<br>Max : {games_no_df.loc[max_count, 'count']}<br>',
    showarrow = True, arrowhead = 3,
    ax = 0, ay = -30,
    bordercolor = 'black', borderwidth = 2, bgcolor = 'green', 
    font_color = 'white', font_weight = 'bold', font_size = 13
)
fig.add_annotation(
    x = games_no_df.loc[min_count, 'Year'],
    y = games_no_df.loc[min_count, 'count'],
    text = f'<br>Min : {games_no_df.loc[min_count, 'count']}<br>',
    showarrow = True, arrowhead = 3,
    ax = 0, ay = 30,
    bordercolor = 'black', borderwidth = 2, bgcolor = 'red', 
    font_color = 'white', font_weight = 'bold', font_size = 13
)
fig.add_vrect(
    x0 = games_no_df.loc[max_inc - 1, 'Year'],
    x1 = games_no_df.loc[max_inc, 'Year'],
    fillcolor = 'lightgreen',
    opacity = 0.5, layer = 'below', line_width = 0, 
    annotation_text=f'Largest Increase {games_no_df.loc[max_inc, 'increase']}', annotation_position='top left',
    annotation_font=dict(size=13, weight='bold', color='black')
)
fig.add_vrect(
    x0 = games_no_df.loc[max_dec - 1, 'Year'],
    x1 = games_no_df.loc[max_dec, 'Year'],
    fillcolor = 'red',
    opacity = 0.25, layer = 'below', line_width = 0, 
    annotation_text=f'Largest Decrease, {games_no_df.loc[max_dec, 'increase']}', annotation_position='bottom right',
    annotation_font=dict(size=13, weight='bold', color='black')
)
fig.update_xaxes(tickmode='array', tickvals=[2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014, 2016, 2018, 2020, 2022, 2024])
fig.show()

#This graph highlights the years with the highest and lowest number of game releases. 
#Periods of sharp increase may be influenced by factors such as technological advancements, improved game development tools, and overall growth in the gaming industry. 
#Conversely, declines may reflect market saturation, economic slowdowns, or shifts in development cycles.

#Notably, the largest increase and decrease both occur around years that represent peak and trough release volumes, 
#suggesting that rapid growth phases are often followed by correction periods in the industry.

In [8]:
#identify the top 3 platforms which benefitted the most during the peak growth period, (2000 to 2001)
#and how that platform has progressed since then until year 2024 by number of games released

case_df = df_exploded.copy()
case_df['Year'] = case_df['releaseDate'].dt.year
peak_df = case_df[case_df['Year'] == 2001]
top3_platform = peak_df.groupby('platform_list')['id'].nunique().nlargest(3)

#get top3 platform in terms of number of games released in 2001, and their progression from 2000 to 2024
#playstation series such as 'PlayStation 1', 'Playstation 3' are grouped together as they belong to same platform group
case_df = case_df[(case_df['Year'] >= 2000) & (case_df['Year'] != 2025)]
games_no_df = case_df[case_df['platform_list'].isin(top3_platform.index)]
games_no_df = games_no_df.groupby(['Year','platform_list'])['id'].nunique().reset_index(name='count')

#get the years in which PC number of games released overtakes PlayStation
pivot_df = games_no_df.pivot(index='Year', columns='platform_list', values='count').fillna(0)
pivot_df['overtake'] = pivot_df['PC'] > pivot_df['PlayStation']
pivot_df['difference'] = pivot_df['PC'] - pivot_df['PlayStation']
pivot_df['difference'] = pivot_df['difference'].astype(int)
overtake_year = pivot_df[pivot_df['overtake'] == True]
overtake_year = overtake_year.reset_index()

fig = px.line(games_no_df, x='Year', y='count', color = 'platform_list', markers=True, 
              title='<b>Progression Trend for Top3 Platforms for No.of Games Released in 2000</b>',
              labels={'count':'No. of Games', 'platform_list':'Platform'})
fig.add_scatter(
    x=overtake_year['Year'],
    y=overtake_year['PC'], 
    mode='markers',
    marker=dict(size=8, color='yellow', line=dict(color='black', width=2)),
    showlegend=False, 
    hoverinfo='skip'
)
#find the year in which the positive difference for PC, between PC and Playstation
max_diff = overtake_year['difference'].idxmax()
fig.add_vline(
    x=overtake_year.loc[max_diff,'Year'], 
    line_width=3, 
    line_dash='dash',
    line_color="#73166F", 
    layer='below',
    annotation_text=f'Max Difference, {overtake_year.loc[max_diff, 'difference']}', 
    annotation_position='top',
    annotation_font=dict(size=12, weight='bold', color='black')
)
fig.update_layout(legend=dict(borderwidth=2, title_text='<b>Platform</b>'))
fig.show()  

#case_df.loc[:,'platform_list'] = case_df['platform_list'].str.replace(r'PlayStation \d+', 'PlayStation', regex=True)

#This graph shows the progression trend for the top 3 platforms in terms of number of games released during the peak period
#Based on this graph, GBA console only had a short period of success, which is followed by progressive declination and discontinuation of the console
#On the other hand, the PlayStation console has performed better than where it began up until 2024 by releasing newer models for their console series
#From year 2001 up until 2011, PlayStation has always had the upper hand against PC, whilst PC dominated in the more recent years

In [9]:
#identify the number of games released by months (for PC, Nintendo, PlayStation and Xbox)
case_df = df_exploded.copy()
case_df['Month Name'] = case_df['releaseDate'].dt.month_name()
case_df['Month'] = case_df['releaseDate'].dt.month
case_df = case_df[case_df['platform_list'].isin(['Nintendo','PlayStation','PC','Xbox'])]
case_df = case_df.groupby(['platform_list','Month','Month Name'])['id'].nunique().reset_index(name='count')
case_df = case_df.sort_values(by='Month', ascending=True)


fig = px.sunburst(case_df, path=['platform_list','Month Name'], values='count', color='platform_list',
                  color_discrete_map={'PlayStation':'blue', 'Nintendo':'red','Xbox':'green','PC':'orange'},
                  title = '<b>No.of Games Released for Top Platforms by Month</b>')

fig.update_traces(textinfo='label+percent entry', insidetextorientation='radial', textfont_size=14,
                  hovertemplate='<b>%{label}</b><br>Percent: %{percentEntry:.2%}<br>Count: %{value}') 
fig.update_layout(height=500)                           
fig.show()


In [10]:
#Identify the number of games released by these 4 platforms in quarterly
case_df = df_exploded.copy()
case_df['Quarter'] = case_df['releaseDate'].dt.quarter
case_df['Quarter_Name'] = case_df['Quarter'].apply(lambda x: f'Q{x}')
case_df['Month'] = case_df['releaseDate'].dt.month
case_df = case_df[case_df['platform_list'].isin(['Nintendo','PlayStation','PC','Xbox'])]
platform_df = case_df.groupby(['platform_list','Month','Quarter_Name'])['id'].nunique().reset_index(name='count')

#find in each platform which quarter has the most number of games released
quarter_df = platform_df.groupby(['platform_list','Quarter_Name'])['count'].sum().reset_index(name='total_games')

#combine the two df
combined_df = platform_df.merge(
    quarter_df[['platform_list','Quarter_Name','total_games']],
    on=['platform_list','Quarter_Name'],
    how='left'
)

combined_df['total_games'] = combined_df['total_games'].fillna(0).astype(int)
quarter_df = quarter_df.loc[quarter_df.groupby('platform_list')['total_games'].idxmax()]
quarter_df = quarter_df.sort_values(by='total_games', ascending=False)

colors = {'PlayStation':'blue', 'Nintendo':'red','Xbox':'green','PC':'orange'}
fig = px.bar(combined_df, x='Quarter_Name', y='count', facet_col='platform_list', color='Quarter_Name', 
             labels={'count':'No.of Games', 'platform_list':'Platform', 'Quarter_Name':'Quarter'}, 
             title='<b>No.of Games Released by Top Platforms in Each Quarter</b>',
             hover_data={'platform_list': True, 'Quarter_Name': True, 'Month': True, 'platform_list': False},
             color_discrete_map=colors)

#manual annotation positions
x_pos = 1.12
y_start = 0.77
y_step = 0.032

for i, row in enumerate(quarter_df.itertuples()):
    fig.add_annotation(
        x=x_pos,
        y=y_start - i*y_step,   
        xref='paper',
        yref='paper',
        text=f"{row.platform_list}: {row.Quarter_Name}-{row.total_games}",
        showarrow=False,
        font=dict(color='black', size=12),
        align='left'
    )

fig.for_each_annotation(lambda x: x.update(text=x.text.replace('Platform=', '')))
fig.update_layout(height=650, width=1400, margin=dict(r=148), legend=dict(borderwidth=1, title_text='<b>Quarter</b>'))
fig.update_xaxes(tickvals=['Q1','Q2','Q3','Q4'], title_text='Quarter')
fig.show()


In [11]:
#line graph to identify trends
colors = {'PlayStation':'blue', 'Nintendo':'red','Xbox':'green','PC':'orange'}
fig = px.line(combined_df, x='Quarter_Name', y='total_games', color='platform_list', markers=True,
              color_discrete_map=colors, labels={'total_games':'Total Games Released','Quarter_Name':'Quarter'},
              title='<b>Total Games Released Trend by Top Platforms in Quarters</b>')

platforms = combined_df['platform_list'].unique()

for platform in platforms:
    df_platform = combined_df[combined_df['platform_list'] == platform]
    
    max_row = df_platform.loc[df_platform['total_games'].idxmax()]
    fig.add_annotation(
        x=max_row['Quarter_Name'],
        y=max_row['total_games'],
        text=f"Max: {max_row['total_games']}",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-45,
        bordercolor = 'black', borderwidth = 1, bgcolor = colors[platform], 
        font_color = 'white', font_weight = 'bold', font_size = 13
    )
    
    min_row = df_platform.loc[df_platform['total_games'].idxmin()]
    fig.add_annotation(
        x=min_row['Quarter_Name'],
        y=min_row['total_games'],
        text=f"Min: {min_row['total_games']}",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=30,
        bordercolor = 'black', borderwidth = 1, bgcolor = colors[platform], 
        font_color = 'white', font_weight = 'bold', font_size = 13
    )

fig.update_layout(legend=dict(title_text='<b>Platform</b>', borderwidth=1))
fig.show()

In [12]:
#identify the game genres / game rating that are popular and well-received in top platforms for the last 5 years (2019 -2024)
#use number of games released for popularity, and user_scores as feedback
case_df = df_exploded.copy()
case_df['Year'] = case_df['releaseDate'].dt.year
case_df = case_df[(case_df['Year'] >= 2019) & (case_df['Year'] < 2025)]
case_df = case_df[case_df['platform_list'].isin(['Nintendo','PlayStation','PC','Xbox'])]
genre_no_df = case_df.groupby('genres')['id'].nunique().nlargest(5).reset_index(name='count')

fig = make_subplots(rows=2, cols=1, specs=[[{'type': 'domain'}],[{'type': 'bar'}]],
                    subplot_titles=('<b>Top Genres in Top Platforms by Games Released (2019-2024)</b>',
                                    '<b>Top Genres in Top Platforms by Userscore (2019-2024)</b>'))

#first graph
fig.add_trace(px.pie(genre_no_df, names='genres', values='count').data[0], row=1, col=1)
fig.update_traces(textinfo='label+percent', textposition='auto',
                  insidetextfont=dict(color='black',size=11, weight='bold'),
                  outsidetextfont=dict(color='black', size=11, weight='bold'), row=1, col=1)

#second graph
genre_avg_df = case_df.groupby(['genres', 'platform_list'], as_index=False)['userscore'].mean()
genre_avg_df['rank'] = genre_avg_df.groupby('platform_list')['userscore'].rank(method='first', ascending=False)
genre_avg_df = genre_avg_df[genre_avg_df['rank'] <= 3].drop(columns='rank')

bar_fig = px.bar(genre_avg_df, x='platform_list', y='userscore', color='genres', barmode='group')
for trace in bar_fig.data:
    fig.add_trace(trace, row=2, col=1)

fig.data[0].legend = 'legend'
for trace in fig.data[1:]:
    trace.legend = 'legend2'

fig.update_annotations(yshift=25)
fig.update_yaxes(title_text='Aveage Userscore', row=2, col=1)
fig.update_layout(height=700, legend=dict(x=1.05, y=0.93, title_text='<b>Genre</b>', borderwidth=1),
                  legend2=dict(x=1.05, y=-0.08, title_text='<b>Genre</b>', borderwidth=1))
fig.show()

In [13]:
case_df = df_exploded.copy()
case_df['Year'] = case_df['releaseDate'].dt.year
case_df = case_df[(case_df['Year'] >= 2019) & (case_df['Year'] < 2025)]
case_df = case_df[case_df['platform_list'].isin(['Nintendo','PlayStation','PC','Xbox'])]

rating_no_df = case_df.groupby('rating')['id'].nunique().reset_index(name='count')
rating_no_df = rating_no_df[rating_no_df['rating'] != 'Rating Pending']
rating_no_df = rating_no_df.nlargest(5, 'count')

fig = make_subplots(rows=2, cols=1, specs=[[{'type': 'domain'}],[{'type': 'bar'}]],
                    subplot_titles=('<b>Top Rating in Top Platforms by Games Released (2019-2024)</b>',
                                    '<b>Top Rating in Top Platforms by Userscore (2019-2024)</b>'))

#first graph
fig.add_trace(px.pie(rating_no_df, names='rating', values='count').data[0], row=1, col=1)
fig.update_traces(textinfo='label+percent', textposition='auto',
                  insidetextfont=dict(color='black',size=13, weight='bold'),
                  outsidetextfont=dict(color='black', size=11, weight='bold'), row=1, col=1)

#second graph
rating_avg_df = case_df.groupby(['rating', 'platform_list'], as_index=False)['userscore'].mean()
rating_avg_df['rank'] = rating_avg_df.groupby('platform_list')['userscore'].rank(method='first', ascending=False)
rating_avg_df = rating_avg_df[rating_avg_df['rank'] <= 3].drop(columns='rank')

bar_fig = px.bar(rating_avg_df, x='platform_list', y='userscore', color='rating', barmode='group')
for trace in bar_fig.data:
    fig.add_trace(trace, row=2, col=1)

fig.data[0].legend = 'legend'
for trace in fig.data[1:]:
    trace.legend = 'legend2'

fig.update_annotations(yshift=25)
fig.update_yaxes(title_text='Aveage Userscore', row=2, col=1)
fig.update_layout(height=700, legend=dict(x=1.05, y=0.93, title_text='<b>Genre</b>', borderwidth=1),
                  legend2=dict(x=1.05, y=0.05, title_text='<b>Genre</b>', borderwidth=1))

In [14]:
#identify platform metascores vs userscore for top genres based on number of games released between year 2019 - 2024 (all platforms)
case_df = df_exploded.copy()
case_df['Year'] = case_df['releaseDate'].dt.year
case_df['score_list'] = case_df['score_list'].astype(int)
case_df = case_df[(case_df['Year'] >= 2019) & (case_df['Year'] < 2025)]
genre_no_df = case_df.groupby('genres')['id'].nunique().nlargest(5).reset_index(name='count')
platform_score_df = case_df[case_df['genres'].isin(genre_no_df['genres'])].groupby('genres')['score_list'].mean().reset_index(name='avg')
platform_score_df['avg'] = platform_score_df['avg'].round(2)

user_score_df = case_df[case_df['genres'].isin(genre_no_df['genres'])].groupby('genres')['userscore'].mean().reset_index(name='avg_user')
user_score_df['avg_user'] = user_score_df['avg_user'].round(2)

combined_df = platform_score_df.merge(user_score_df, on='genres')
combined_df = combined_df.melt(id_vars='genres', var_name='Score Type', value_vars=['avg','avg_user'], value_name='Score')
combined_df['Score Type'] = combined_df['Score Type'].replace({'avg': 'Platform Metascore', 'avg_user': 'User Score'})

fig = px.bar(combined_df, x='genres', y='Score', text_auto='.2f', color='genres', text='Score', facet_col='Score Type',
             labels={'Score':'Average Score', 'genres':'Genre'}, title='<b>Platform Metascores vs Userscore for Top Genres (2019-2024)</b>')
fig.update_yaxes(range=[0, 100])
fig.update_traces(textposition='inside', textfont_size=15, textfont_color='white', textfont_weight='bold')
fig.for_each_annotation(lambda x: x.update(text=x.text.replace('Score Type=','')))
fig.update_layout(legend=dict(y=0.9, borderwidth=1, title_text='<b>Genre</b>'))
fig.show()

combined_df

,genres,Score Type,Score
0,2D Platformer,Platform Metascore,74.98
1,Action Adventure,Platform Metascore,72.79
2,Action RPG,Platform Metascore,72.97
3,Adventure,Platform Metascore,75.39
4,Open-World Action,Platform Metascore,72.43
5,2D Platformer,User Score,65.69
6,Action Adventure,User Score,65.18
7,Action RPG,User Score,66.02
8,Adventure,User Score,62.88
9,Open-World Action,User Score,65.50


In [15]:
#identify metascore sentiment against userscore sentiment distribution in overall top platforms(2014-2024)
#identify average metascore sentiment vs userscore sentiment across the 10 years

case_df = df_exploded.copy()
case_df['Year'] = case_df['releaseDate'].dt.year
case_df = case_df[(case_df['Year'] >= 2014) & (case_df['Year'] < 2025)]
case_df = case_df[case_df['platform_list'].isin(['Nintendo','PlayStation','PC','Xbox'])]
userscore_df = case_df.groupby('userscore_sentiment')['id'].nunique().reset_index(name='user_count')
metascore_df = case_df.groupby('metascore_sentiment')['id'].nunique().reset_index(name='meta_count') 
userscore_df  = userscore_df.rename(columns={'userscore_sentiment':'sentiment'})
metascore_df = metascore_df.rename(columns={'metascore_sentiment':'sentiment'})

combined_df = userscore_df.merge(metascore_df, on='sentiment')
combined_df['diff'] = (combined_df['user_count'] - combined_df['meta_count']).abs()

combined_df = combined_df.melt(id_vars='sentiment', var_name='count_criteria', value_vars=['user_count','meta_count', 'diff'], value_name='value')
combined_df['count_criteria'] = combined_df['count_criteria'].replace({'user_count':'No.of Userscore', 'meta_count':'No.of Metascore', 
                                                                       'diff':'Userscore vs Metascore'})

fig = px.bar(combined_df, x='sentiment', y='value', color='sentiment', facet_col='count_criteria',
             title='<b>Metascore and Userscore Distribution in Top Platforms((2014-2024)</b>')

fig.for_each_annotation(lambda x: x.update(text=x.text.replace('count_criteria=','')))
fig.update_xaxes(title_text='')


fig.update_layout(legend=dict(title_text='<b>Criteria</b>', borderwidth=1))


fig.show()




In [16]:
case_df = df_exploded.copy()
case_df['Year'] = case_df['releaseDate'].dt.year
case_df = case_df[(case_df['Year'] >= 2014) & (case_df['Year'] < 2025)]
case_df = case_df[case_df['platform_list'].isin(['Nintendo','PlayStation','PC','Xbox'])]
meta_avg_df = case_df.groupby('Year')['metascore'].mean().reset_index(name='meta_avg')
meta_avg_df['meta_avg'] = meta_avg_df['meta_avg'].round(2)
user_avg_df = case_df.groupby('Year')['userscore'].mean().reset_index(name='user_avg')
user_avg_df['user_avg'] = user_avg_df['user_avg'].round(2)

combined_df = meta_avg_df.merge(user_avg_df, on='Year')
combined_df['diff'] = (combined_df['meta_avg'] - combined_df['user_avg']).abs()

combined_df = combined_df.melt(id_vars='Year', var_name='criteria', value_vars=['meta_avg','user_avg','diff'], value_name='value')
combined_df['criteria'] = combined_df['criteria'].replace({'meta_avg':'Average of Metascore', 'user_avg':'Average of Userscore',
                                                          'diff':'Userscore vs Metascore'})
plot_df = combined_df[combined_df['criteria'].isin(['Average of Metascore','Average of Userscore'])]

fig = px.line(plot_df, x='Year', y='value', color='criteria', markers=True, labels={'value':'Average Score'},
               title='<b>Average of Metascore and Userscore in Top Platforms(2014-2024)</b>')

diff_df = combined_df[combined_df['criteria'] == 'Userscore vs Metascore']
max_diff = diff_df['value'].idxmax()
min_diff = diff_df['value'].idxmin()

fig.add_vline(
    x=diff_df.loc[max_diff,'Year'], 
    line_width=5, 
    line_dash='longdash',
    line_color="#BC1C1C", 
    layer='below',
    annotation_text=f'Max Difference, {diff_df.loc[max_diff, 'value']:.2f}', 
    annotation_position='top',
    annotation_font=dict(size=12, weight='bold', color='black')
)
fig.add_vline(
    x=diff_df.loc[min_diff,'Year'], 
    line_width=5, 
    line_dash='longdash',
    line_color="#84C544", 
    layer='below',
    annotation_text=f'Min Difference, {diff_df.loc[min_diff, 'value']:.2f}', 
    annotation_position='top',
    annotation_font=dict(size=12, weight='bold', color='black')
)

fig.update_layout(legend=dict(title_text='<b>Criteria</b>', borderwidth=1))
fig.update_xaxes(tickmode='array', tickvals=[2014, 2016, 2018, 2020, 2022, 2024])
fig.show()
